# Model 3: Predict Star Formation Variables (sSFR, SFR, M*, age) Based on Visual Morphology (galaxy image)


In [17]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,r2_score
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL
import time 

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True

%matplotlib qt




[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data from Schema Table

In [18]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)


galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 

galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)


# Importing Images from their Downloaded Locations 


In [19]:
# image_locations=[]
# for i in range (len(galaxy_list)):
#     image_locations.append(Image(galaxy_list[i]).filename)
    
# image_locations=np.array(image_locations,dtype=str)
# np.savetxt('Image Directories',image_locations,fmt='%s')

image_locations=np.loadtxt('Image Directories',dtype=str)



In [20]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    from PIL import Image, ImageOps 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image


# Putting the Images into DataLoaders

In [21]:
img_size=(128,128)

# image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/') #Picks up 3590 pictures, directory list has lenght 3637 however 

image_directory='/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3'
classes= sorted(os.listdir(image_directory))

batch_size=50
image_copies=30

def load_dataset(data_path):
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        #Rotates the image by some angle
        transforms.RandomRotation(0,360),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        # transform=transformation
    )
    
    #This loop transforms the images and assigns them the correct label 

    full_dataset_v2 = []   
    for i in range(len(full_dataset)): 
        for j in range(image_copies): #This loops makes it so we have 5 copies of each galaxy image with different orientations 
            temp=transformation(resize_image(full_dataset[i][0]))
            full_dataset_v2.append((temp,log_SFR.iloc[i])) 
    
    full_dataset=full_dataset_v2

    print(len(full_dataset))


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader




#####################################################################################################



# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(image_directory)
batch_size = train_loader.batch_size
print("Data loaders ready to read", image_directory)



107670
Data loaders ready to read /home/juanp/sas/dr15/manga/spectro/redux/v2_4_3


In [22]:
print(ImageFolder(image_directory))
print(len(image_locations))
print(len(np.unique(image_locations)))
print(len(np.unique(galaxy_list)))




# full_dataset = torchvision.datasets.ImageFolder(
#         root='/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/7968',
#         # transform=transformation
#     ) 

# full_dataset_v2 = []   
# for i in range(len(full_dataset)): 
#     full_dataset_v2.append((resize_image(full_dataset[i][0]) , i)) 

# print(full_dataset_v2)


# print(full_dataset[0][1])
# for i in range(len(full_dataset)): 
#         full_dataset[i] = (resize_image(full_dataset[i][0]) , 0) 
        # full_dataset[i][0] = resize_image(full_dataset[i][0])
        # full_dataset[i][1] = correct_label(correct_manga_id) 

Dataset ImageFolder
    Number of datapoints: 3589
    Root location: /home/juanp/sas/dr15/manga/spectro/redux/v2_4_3
3637
3589
3589


# Defining the Model 

In [23]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 128 x 128, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=10)

        self.fc1= nn.Linear(in_features=10, out_features=1)


    def forward(self, x):
        # x=x+1
        
        # In the forward function, pass the data through the layers we defined in the init function
        # print(x)
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x=self.conv1(x)
        
        x=self.pool(x)
       

        x=F.relu(x)

        
      
 
     
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x))) 
        
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        
        # Return class probabilities via a log_softmax function
        x=F.relu(x) 
        
        x= self.fc1(x)
        # print(x)
        return x
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=len(SFR)).to('cuda')

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
  (fc1): Linear(in_features=10, out_features=1, bias=True)
)


# Creating Training Loop/Function

In [24]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):

        target=target.float()
        # print(target.shape)
        target=target.reshape(-1,1)
        # Use the CPU or GPU as appropriate
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
    
        output = model(data)

        
        # Get the loss
        loss = loss_criteria(output, target)
        if batch_idx==0:
            print(output,target,output-target,loss)
        # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()

       
        
        # Print metrics so we see some progress
        # print('\tTraining batch {} Loss: {:.6f}'.format(batch_idx + 1, loss.item()))
            
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

# Create Test Function

In [25]:
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            target=target.float()
            target=target.reshape(-1,1)
            batch_count += 1
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return avg_loss

# Training the Model

In [26]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.MSELoss(reduction='sum')

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []


# Train over 10 epochs (We restrict to 10 for time issues)
epochs = 1000
print('Training on', device)
time_start = time.time()
for epoch in range(1, epochs + 1):
        print(epoch)
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)
time_end = time.time()
total_time = (time_end-time_start)/60 
print(total_time)

 [-0.3613],
        [ 0.0553],
        [-0.4980],
        [ 0.1164],
        [ 0.2100],
        [ 0.0040],
        [ 0.0481],
        [ 0.3241],
        [ 0.2020],
        [ 0.7370],
        [ 0.3017],
        [ 0.3843],
        [ 0.0287],
        [ 0.3683],
        [-0.2993],
        [ 0.4699],
        [ 0.5494],
        [-0.2735],
        [-0.2666],
        [ 0.4734],
        [ 0.2067],
        [-0.0084],
        [ 0.1948],
        [ 1.3919],
        [ 0.2183],
        [ 0.3202],
        [-0.1342],
        [-0.1058],
        [ 0.9476],
        [ 0.1726],
        [-0.3720],
        [-0.9110],
        [ 0.0494],
        [ 0.1516],
        [ 0.2054],
        [ 0.1503],
        [ 0.1305],
        [-0.0635],
        [ 0.3998],
        [-0.0661],
        [ 0.6653],
        [ 0.2341],
        [ 0.1322]], device='cuda:0', grad_fn=<SubBackward0>) tensor(8.0941, device='cuda:0', grad_fn=<MseLossBackward>)
Training set: Average loss: 6.898967
Validation set: Average loss: 3.114713, Accuracy: 0/

# Checking How Well the Model is Preforming 

In [27]:
all_truths_train = [] 
all_preds_train = [] 
for (data,target) in train_loader:
    data, target = data.to(device), target.to(device)
    output=model(data)
    all_truths_train.append(target.cpu().detach().numpy())
    all_preds_train.append(output.cpu().detach().numpy()) 


incomplete_batch_id_train=len(all_truths_train)-1

remainder_train=len(all_truths_train[incomplete_batch_id_train])

total_values_train=(len(all_truths_train)*batch_size)-(batch_size-remainder_train)



all_truths_train_array=np.zeros(total_values_train)
all_preds_train_array=np.zeros(total_values_train)
k=0
while k < total_values_train:
    for i in range(len(all_truths_train)):
        if i<incomplete_batch_id_train:
            for j in range(batch_size):
                all_truths_train_array[k]=all_truths_train[i][j]
                all_preds_train_array[k]=all_preds_train[i][j]
                k=k+1
                


        else:
            i=incomplete_batch_id_train
            for j in range(remainder_train):
                all_truths_train_array[k]=all_truths_train[i][j]
                all_preds_train_array[k]=all_preds_train[i][j]
                k=k+1
                



# all_truths_train=all_truths_train_array
# all_preds_train=all_preds_train_array


In [28]:
all_truths_test = [] 
all_preds_test = [] 
for (data,target) in test_loader:
    data, target = data.to(device), target.to(device)
    output=model(data)
    all_truths_test.append(target.cpu().detach().numpy())
    all_preds_test.append(output.cpu().detach().numpy()) 


incomplete_batch_id_test=len(all_truths_test)-1

remainder_test=len(all_truths_test[incomplete_batch_id_test])


total_values_test=(len(all_truths_test)*batch_size)-(batch_size-remainder_test)



all_truths_test_array=np.zeros(total_values_test)
all_preds_test_array=np.zeros(total_values_test)
k=0
while k < total_values_test:
    for i in range(len(all_truths_test)):
        if i<incomplete_batch_id_test:
            for j in range(batch_size):
                all_truths_test_array[k]=all_truths_test[i][j]
                all_preds_test_array[k]=all_preds_test[i][j]
                # print(i,j,k)
                k=k+1
                
                


        else:
            i=incomplete_batch_id_test
            for j in range(remainder_test):
                all_truths_test_array[k]=all_truths_test[i][j]
                all_preds_test_array[k]=all_preds_test[i][j]
                # print(i,j,k)
                k=k+1
                
                

print(all_truths_test[3][43])
print(all_preds_test[3][43])
print(all_truths_test_array[193])
print(all_preds_test_array[193])
# all_truths_test=all_truths_test_array
# all_preds_test=all_preds_test_array



-1.7977967652124356
[-1.5908219]
-1.7977967652124356
-1.5908218622207642


In [29]:
r2_train=r2_score(all_truths_train_array,all_preds_train_array)
r2_test=r2_score(all_truths_test_array,all_preds_test_array)
plt.figure(figsize=(16,12))
plt.subplot(1,2,1) 
plt.title('Test Data Set')
plt.scatter(all_truths_test_array, all_preds_test_array,color = 'b', alpha = 0.1*7/3)
plt.xlabel('Test True Values')
plt.ylabel('Test Predicted Value')
plt.text(0,-5,'R$^2$='+ str(round(r2_test,4)))
plt.plot([-5,1],[-5,1],'r--')

plt.subplot(1,2,2)
plt.title('Train Data Set')
plt.scatter(all_truths_train_array, all_preds_train_array,color = 'k', alpha = 0.1)
plt.xlabel('Train True Values')
plt.ylabel('Train Predicted Value')
plt.plot([-5,1],[-5,1],'r--') 
plt.text(0,-5.5,'R$^2$='+str(round(r2_train,3)))
plt.show()
plt.savefig('/home/juanp/Documents/SURP-2021/Plots/Model 3/Rotations 30 Copies of Images.pdf')

In [30]:
torch.save(model.state_dict(),'/home/juanp/Documents/SURP-2021/Models/SFR_Model_Rotation_30_Copies_Images')

# Loss as a Function of Epoch

In [31]:


# plt.figure(figsize=(15,15))
# plt.plot(epoch_nums, np.log10(training_loss))
# plt.plot(epoch_nums, np.log10(validation_loss))
# plt.xlabel('epoch')
# plt.ylabel('Log of loss')
# plt.legend(['training', 'validation'], loc='upper right')
# plt.show()



# Plotting Galaxy Images and their True SFR vs CNN Predicted SFR

In [32]:
# plt.figure(figsize=(12,4))
#     plt.subplot(1,3,1)
#     plt.imshow(picture[0][0,0:,0:])
#     plt.subplot(1,3,2)
#     plt.imshow(picture[1][0,0:,0:])
#     plt.subplot(1,3,3)
#     plt.imshow(picture[2][0,0:,0:])
#     plt.show()